In [1]:
import os
import gc
import tqdm
import json
import ray
import torch
import shutil
import numpy as np
from transformers import AutoTokenizer

In [2]:
ray.init()

2020-03-04 13:29:16,418	WARNING services.py:597 -- setpgrp failed, processes may not be cleaned up properly: [Errno 1] Operation not permitted.
2020-03-04 13:29:16,419	INFO resource_spec.py:216 -- Starting Ray with 35.4 GiB memory available for workers and up to 17.72 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).


{'node_ip_address': '169.237.10.101',
 'redis_address': '169.237.10.101:53016',
 'object_store_address': '/tmp/ray/session_2020-03-04_13-29-16_417372_120662/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-03-04_13-29-16_417372_120662/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2020-03-04_13-29-16_417372_120662'}

## Use the tokenizer you want

In [3]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

In [4]:
TOTAL_NUM_SECTORS = 128
NUM_THREADS = 128

In [5]:
os.makedirs("cache", exist_ok=True)

### DATA_PATHSplit files into multiple files

In [6]:
f_writes = [open(os.path.join("cache", str(i) + '.jsonl'), "w") for i in range(NUM_THREADS)]

with open("train_data.jsonl") as f_read:
    for i, line in enumerate(tqdm.tqdm(f_read)):
        f_writes[i%NUM_THREADS].write(line)
                 
[f_writes[i].close() for i in range(NUM_THREADS)]

12851539it [01:42, 124913.47it/s]


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

### Process Data

In [7]:
DATA_PATH = "SIMPLE_DATA"
os.makedirs(DATA_PATH, exist_ok=True)

In [8]:
@ray.remote
def process_data(tokenizer, rank):
    with open(f"cache/{rank}.jsonl", "r") as f_read:
        with open(f"cache/{rank}_1.jsonl", "w") as f_write:
            if rank==0:
                f_read = tqdm.tqdm(f_read)

            for line in f_read:
                text = json.loads(line)["text"].replace("\x00", "")
                tokens = tokenizer.tokenize(text)
                ids = tokenizer.convert_tokens_to_ids(tokens)
                line = json.dumps(ids)
                f_write.write(line)
                f_write.write("\n")
            
    return 0

In [9]:
ray_objs = []

for rank in range(NUM_THREADS):
    ray_obj = process_data.remote(tokenizer, rank=rank)
    ray_objs.append(ray_obj)
    
for rank in tqdm.trange(NUM_THREADS):
    assert ray.get(ray_objs[rank]) == 0

0it [00:00, ?it/s]43) 
7it [00:00, 64.68it/s] 
13it [00:00, 60.62it/s]0m 
25it [00:00, 70.57it/s]0m 
39it [00:00, 80.44it/s]0m 
50it [00:00, 85.14it/s]0m 
59it [00:00, 64.80it/s]0m 
66it [00:00, 60.71it/s]0m 
78it [00:01, 70.12it/s]0m 
86it [00:01, 69.51it/s]0m 
94it [00:01, 65.95it/s]0m 
102it [00:01, 67.20it/s]m 
111it [00:01, 72.01it/s]m 
121it [00:01, 78.15it/s]m 
131it [00:01, 79.96it/s]m 
140it [00:01, 61.45it/s]m 
150it [00:02, 69.39it/s]m 
158it [00:02, 68.36it/s]m 
166it [00:02, 40.38it/s]m 
177it [00:02, 49.54it/s]m 
188it [00:02, 58.63it/s]m 
199it [00:02, 67.94it/s]m 
208it [00:03, 65.53it/s]m 
223it [00:03, 77.88it/s]m 
234it [00:03, 84.98it/s]m 
245it [00:03, 86.40it/s]m 
261it [00:03, 96.01it/s]m 
274it [00:03, 104.12it/s] 
286it [00:03, 102.44it/s] 
297it [00:04, 31.53it/s]  
306it [00:04, 36.31it/s]m 
319it [00:04, 46.25it/s]m 
329it [00:05, 49.96it/s]m 
340it [00:05, 59.60it/s]m 
355it [00:05, 72.64it/s]m 
368it [00:05, 82.01it/s]m 
380it [00:05, 88.93it/s]m 
392it [0

  0%|          | 0/128 [00:00<?, ?it/s]

584it [00:07, 124.88it/s] 
598it [00:08, 34.16it/s]  
612it [00:08, 44.10it/s]m 
627it [00:08, 55.05it/s]m 
640it [00:08, 66.02it/s]m 
652it [00:09, 47.31it/s]m 
666it [00:09, 58.97it/s]m 
678it [00:09, 62.84it/s]m 
693it [00:09, 75.26it/s]m 
714it [00:09, 93.11it/s]m 
728it [00:09, 63.13it/s]m 
743it [00:10, 74.43it/s]m 
755it [00:10, 79.10it/s]m 
779it [00:10, 97.74it/s]m 
794it [00:10, 101.10it/s] 
810it [00:10, 107.62it/s] 
825it [00:10, 111.74it/s] 
838it [00:10, 88.85it/s]  
852it [00:11, 99.00it/s]m 
864it [00:11, 65.31it/s]m 
874it [00:11, 56.97it/s]m 
892it [00:11, 71.16it/s]m 
909it [00:11, 86.07it/s]m 
925it [00:11, 98.90it/s]m 
939it [00:12, 102.60it/s] 
956it [00:12, 112.24it/s] 
975it [00:12, 127.53it/s] 
990it [00:12, 118.74it/s] 
1005it [00:12, 124.48it/s]
1019it [00:12, 119.96it/s]
1037it [00:12, 131.08it/s]
1055it [00:12, 125.69it/s]
1069it [00:13, 128.76it/s]
1091it [00:13, 146.94it/s]
1107it [00:13, 139.44it/s]
1122it [00:13, 138.40it/s]
1139it [00:13, 146.40it/s]
1

5721it [00:55, 121.90it/s]
5744it [00:55, 141.81it/s]
5761it [00:55, 139.92it/s]
5783it [00:55, 156.72it/s]
5801it [00:56, 114.06it/s]
5821it [00:56, 130.17it/s]
5845it [00:56, 148.77it/s]
5863it [00:56, 134.94it/s]
5883it [00:56, 144.19it/s]
5904it [00:56, 158.35it/s]
5922it [00:56, 157.67it/s]
5941it [00:57, 165.81it/s]
5959it [00:57, 159.48it/s]
5976it [00:57, 151.53it/s]
5992it [00:57, 148.47it/s]
6018it [00:57, 163.19it/s]
6036it [00:57, 166.02it/s]
6058it [00:57, 174.98it/s]
6076it [00:57, 143.28it/s]
6092it [00:58, 109.32it/s]
6111it [00:58, 124.50it/s]
6136it [00:58, 134.01it/s]
6154it [00:58, 142.36it/s]
6170it [00:58, 138.50it/s]
6187it [01:00, 22.05it/s] 
6202it [01:01, 29.62it/s] 
6221it [01:01, 39.62it/s] 
6239it [01:01, 51.58it/s] 
6254it [01:02, 29.82it/s] 
6265it [01:02, 37.37it/s] 
6278it [01:02, 47.37it/s] 
6291it [01:02, 55.33it/s] 
6307it [01:02, 68.71it/s] 
6325it [01:02, 84.12it/s] 
6343it [01:02, 100.07it/s]
6359it [01:03, 94.04it/s] 
6375it [01:03, 106.75it/s]
6

11349it [01:47, 152.44it/s]
11371it [01:47, 163.38it/s]
11389it [01:48, 158.32it/s]
11406it [01:48, 149.20it/s]
11422it [01:48, 135.47it/s]
11437it [01:48, 135.32it/s]
11451it [01:48, 132.91it/s]
11465it [01:48, 119.85it/s]
11487it [01:48, 137.25it/s]
11511it [01:48, 157.42it/s]
11533it [01:49, 171.51it/s]
11552it [01:49, 155.49it/s]
11580it [01:49, 179.24it/s]
11601it [01:49, 158.00it/s]
11627it [01:49, 178.28it/s]
11648it [01:49, 174.74it/s]
11668it [01:49, 172.85it/s]
11689it [01:49, 181.35it/s]
11709it [01:50, 59.35it/s] 
11734it [01:50, 76.75it/s]
11755it [01:50, 94.54it/s]
11774it [01:51, 106.61it/s]
11792it [01:51, 117.92it/s]
11810it [01:51, 128.54it/s]
11827it [01:51, 134.81it/s]
11844it [01:51, 124.78it/s]
11863it [01:51, 137.77it/s]
11879it [01:51, 120.67it/s]
11907it [01:51, 145.24it/s]
11932it [01:52, 166.09it/s]
11953it [01:52, 166.13it/s]
11977it [01:52, 179.52it/s]
11997it [01:52, 157.50it/s]
12017it [01:52, 165.02it/s]
12035it [01:52, 142.69it/s]
12051it [01:52, 142.35

...

82146it [11:27, 198.99it/s]
82167it [11:27, 195.16it/s]
82187it [11:28, 167.08it/s]
82205it [11:28, 140.80it/s]
82230it [11:29, 60.45it/s] 
82258it [11:29, 78.59it/s]
82283it [11:29, 97.08it/s]
82305it [11:29, 115.39it/s]
82325it [11:29, 128.80it/s]
82344it [11:29, 134.62it/s]
82365it [11:29, 149.59it/s]
82384it [11:30, 95.85it/s] 
82410it [11:30, 117.01it/s]
82428it [11:30, 120.58it/s]
82452it [11:30, 134.09it/s]
82469it [11:30, 142.53it/s]
82488it [11:30, 153.09it/s]
82506it [11:30, 157.72it/s]
82530it [11:31, 175.36it/s]
82550it [11:31, 181.06it/s]
82570it [11:31, 166.91it/s]
82590it [11:31, 171.82it/s]
82608it [11:33, 26.97it/s] 
82628it [11:33, 33.57it/s]
82640it [11:33, 41.06it/s]
82665it [11:33, 54.34it/s]
82679it [11:34, 66.20it/s]
82705it [11:34, 83.49it/s]
82722it [11:34, 92.12it/s]
82750it [11:34, 115.03it/s]
82774it [11:34, 105.53it/s]
82793it [11:34, 120.66it/s]
82810it [11:35, 67.29it/s] 
82833it [11:35, 85.07it/s]
82849it [11:35, 94.94it/s]
82873it [11:35, 115.20it/s]
82

88285it [12:18, 133.96it/s]
88302it [12:19, 43.19it/s] 
88317it [12:20, 54.32it/s]
88330it [12:20, 63.91it/s]
88342it [12:20, 72.51it/s]
88369it [12:20, 91.72it/s]
88384it [12:20, 59.25it/s]
88406it [12:21, 71.71it/s]
88426it [12:21, 88.74it/s]
88446it [12:21, 103.04it/s]
88469it [12:21, 122.79it/s]
88487it [12:21, 129.54it/s]
88505it [12:21, 139.77it/s]
88526it [12:21, 153.79it/s]
88544it [12:21, 151.93it/s]
88569it [12:21, 172.00it/s]
88589it [12:22, 157.14it/s]
88607it [12:22, 71.17it/s] 
88635it [12:22, 91.06it/s]
88656it [12:22, 109.66it/s]
88675it [12:22, 125.34it/s]
88699it [12:23, 143.44it/s]
88720it [12:23, 157.34it/s]
88740it [12:23, 145.46it/s]
88758it [12:23, 144.38it/s]
88778it [12:23, 156.16it/s]
88798it [12:23, 161.87it/s]
88822it [12:23, 178.14it/s]
88848it [12:23, 195.08it/s]
88869it [12:24, 165.03it/s]
88888it [12:24, 155.40it/s]
88912it [12:24, 171.75it/s]
88931it [12:24, 173.15it/s]
88950it [12:24, 171.13it/s]
88970it [12:24, 156.83it/s]
88995it [12:24, 174.51it/s]


94212it [13:08, 150.73it/s]
94229it [13:08, 115.00it/s]
94244it [13:08, 121.79it/s]
94259it [13:09, 36.27it/s] 
94281it [13:09, 48.38it/s]
94299it [13:09, 60.50it/s]
94316it [13:09, 74.78it/s]
94338it [13:09, 93.11it/s]
94370it [13:09, 115.70it/s]
94390it [13:10, 129.33it/s]
94410it [13:10, 123.93it/s]
94435it [13:10, 145.41it/s]
94455it [13:10, 157.54it/s]
94475it [13:10, 168.07it/s]
94495it [13:10, 172.58it/s]
94522it [13:10, 192.75it/s]
94544it [13:10, 194.89it/s]
94565it [13:11, 187.31it/s]
94586it [13:11, 191.83it/s]
94608it [13:11, 198.05it/s]
94629it [13:11, 194.22it/s]
94651it [13:11, 199.52it/s]
94675it [13:11, 208.52it/s]
94697it [13:11, 196.71it/s]
94718it [13:11, 196.77it/s]
94741it [13:11, 204.94it/s]
94762it [13:11, 205.31it/s]
94786it [13:12, 214.09it/s]
94808it [13:12, 206.15it/s]
94831it [13:12, 211.71it/s]
94853it [13:12, 197.17it/s]
94874it [13:12, 165.99it/s]
94894it [13:12, 174.41it/s]
94914it [13:12, 180.42it/s]
94939it [13:12, 194.40it/s]
94960it [13:13, 190.52it

100371it [13:51, 172.28it/s]
100403it [13:52, 120.66it/s]


100%|██████████| 128/128 [1:00:36<00:00, 28.41s/it] 


In [10]:
len(ray_objs)

128

## Write into final data

In [11]:
for i in tqdm.trange(NUM_THREADS):
    data = []
    
    with open(os.path.join("cache", f"{i}_1.jsonl"), 'r') as f_read:
        for line in tqdm.tqdm(f_read):
            line = torch.IntTensor(json.loads(line))
            data.append(line)

    torch.save(data, os.path.join(DATA_PATH, f"{i}.pkl"))

  0%|          | 0/128 [00:00<?, ?it/s]
0it [00:00, ?it/s]
425it [00:00, 2322.00it/s]
1484it [00:00, 3032.14it/s]
2474it [00:00, 3796.52it/s]
3065it [00:00, 4245.35it/s]
4172it [00:00, 5206.08it/s]
4955it [00:00, 5773.36it/s]
6028it [00:00, 6699.52it/s]
6863it [00:00, 6552.65it/s]
7639it [00:01, 6851.85it/s]
8451it [00:01, 7147.35it/s]
9467it [00:01, 7844.60it/s]
10343it [00:01, 8096.53it/s]
11469it [00:01, 8839.96it/s]
12572it [00:01, 9398.38it/s]
13569it [00:01, 9562.67it/s]
14559it [00:01, 9652.12it/s]
15548it [00:01, 9691.70it/s]
16800it [00:01, 10391.46it/s]
17917it [00:02, 10612.85it/s]
18998it [00:02, 10089.97it/s]
20026it [00:02, 8867.57it/s] 
21352it [00:02, 9843.73it/s]
22502it [00:02, 10286.93it/s]
23580it [00:02, 9994.53it/s] 
24616it [00:02, 9746.78it/s]
25618it [00:02, 9789.03it/s]
26616it [00:02, 8925.85it/s]
27537it [00:03, 7727.89it/s]
28499it [00:03, 8028.07it/s]
29499it [00:03, 8296.06it/s]
30636it [00:03, 9027.74it/s]
31578it [00:03, 8549.69it/s]
32486it [00:03, 869

61575it [00:06, 8986.67it/s]
62737it [00:06, 9641.34it/s]
63792it [00:06, 9896.88it/s]
64921it [00:06, 10276.73it/s]
65968it [00:06, 10215.02it/s]
67179it [00:06, 10714.30it/s]
68267it [00:07, 10274.47it/s]
69310it [00:07, 9535.71it/s] 
70285it [00:07, 9450.52it/s]
71245it [00:07, 8584.08it/s]
72573it [00:07, 9599.32it/s]
73590it [00:07, 9639.04it/s]
74827it [00:07, 10305.32it/s]
75899it [00:07, 9566.88it/s] 
76895it [00:08, 8290.06it/s]
77782it [00:08, 8274.31it/s]
78920it [00:08, 8999.12it/s]
79996it [00:08, 9463.46it/s]
81202it [00:08, 9931.51it/s]
82503it [00:08, 10690.32it/s]
83833it [00:08, 11358.64it/s]
85009it [00:08, 10861.35it/s]
86128it [00:08, 9837.05it/s] 
87161it [00:08, 9979.49it/s]
88189it [00:09, 10021.11it/s]
89229it [00:09, 10127.47it/s]
90339it [00:09, 10145.59it/s]
91364it [00:09, 9865.59it/s] 
92419it [00:09, 10058.82it/s]
93442it [00:09, 10109.49it/s]
94467it [00:09, 10055.89it/s]
95477it [00:09, 9990.80it/s] 
96800it [00:09, 10781.22it/s]
97935it [00:10, 10945.2

25745it [00:02, 11223.01it/s]
26887it [00:02, 10450.58it/s]
27957it [00:02, 10270.04it/s]
29002it [00:02, 10128.10it/s]
30283it [00:02, 10805.32it/s]
31539it [00:02, 11277.61it/s]
32688it [00:03, 11111.49it/s]
33885it [00:03, 11355.76it/s]
35033it [00:03, 10132.90it/s]
36080it [00:03, 10103.93it/s]
37114it [00:03, 9214.01it/s] 
38067it [00:03, 8975.02it/s]
39073it [00:03, 9274.99it/s]
40340it [00:03, 10080.08it/s]
41549it [00:03, 10538.58it/s]
42815it [00:04, 11095.89it/s]
43953it [00:04, 10830.40it/s]
45188it [00:04, 11242.38it/s]
46473it [00:04, 11678.23it/s]
47659it [00:04, 11379.18it/s]
48811it [00:04, 11033.79it/s]
49927it [00:04, 10498.82it/s]
50991it [00:04, 9561.62it/s] 
51974it [00:04, 8943.19it/s]
53150it [00:05, 9635.20it/s]
54148it [00:05, 8655.41it/s]
55185it [00:05, 8985.35it/s]
56117it [00:05, 8955.81it/s]
57348it [00:05, 9752.66it/s]
58549it [00:05, 10311.63it/s]
59615it [00:05, 10194.14it/s]
60948it [00:05, 10960.05it/s]
62078it [00:05, 10905.35it/s]
63288it [00:06, 11

92080it [00:09, 8628.33it/s]
92984it [00:09, 7730.39it/s]
94069it [00:09, 8458.93it/s]
94968it [00:10, 8242.02it/s]
95881it [00:10, 8355.69it/s]
96815it [00:10, 8627.63it/s]
97702it [00:10, 8697.74it/s]
98865it [00:10, 9406.39it/s]
100403it [00:10, 9418.74it/s][A
  6%|▋         | 8/128 [01:47<26:47, 13.40s/it]
0it [00:00, ?it/s]
1105it [00:00, 9397.52it/s]
2216it [00:00, 9853.12it/s]
3448it [00:00, 9931.08it/s]
4590it [00:00, 10334.57it/s]
5641it [00:00, 10234.24it/s]
6909it [00:00, 10862.74it/s]
8102it [00:00, 11161.71it/s]
9264it [00:00, 11290.73it/s]
10348it [00:00, 10499.31it/s]
11401it [00:01, 10507.99it/s]
12582it [00:01, 10865.69it/s]
13662it [00:01, 9180.94it/s] 
14854it [00:01, 9858.58it/s]
15882it [00:01, 9931.27it/s]
16905it [00:01, 8801.22it/s]
18234it [00:01, 9791.96it/s]
19280it [00:01, 9749.84it/s]
20315it [00:01, 9902.71it/s]
21339it [00:02, 8812.16it/s]
22524it [00:02, 9545.79it/s]
23734it [00:02, 10190.93it/s]
24900it [00:02, 10432.52it/s]
25978it [00:02, 10133.47it/s

58580it [00:05, 10837.82it/s]
59942it [00:06, 11542.78it/s]
61144it [00:06, 11680.24it/s]
62369it [00:06, 11834.62it/s]
63565it [00:06, 10824.00it/s]
64673it [00:06, 9894.91it/s] 
65696it [00:06, 9945.38it/s]
66714it [00:06, 9458.86it/s]
67682it [00:06, 9257.18it/s]
68624it [00:06, 9042.92it/s]
69845it [00:07, 9805.45it/s]
70853it [00:07, 7957.78it/s]
71725it [00:07, 8028.00it/s]
72841it [00:07, 8761.76it/s]
73792it [00:07, 8972.12it/s]
75049it [00:07, 9810.73it/s]
76083it [00:07, 9959.28it/s]
77117it [00:07, 9006.50it/s]
78061it [00:08, 8705.53it/s]
79303it [00:08, 9561.00it/s]
80308it [00:08, 9068.93it/s]
81407it [00:08, 9528.21it/s]
82462it [00:08, 9805.80it/s]
83469it [00:08, 8762.01it/s]
84449it [00:08, 9049.29it/s]
85640it [00:08, 9749.42it/s]
86651it [00:08, 9807.68it/s]
87803it [00:08, 10262.54it/s]
88853it [00:09, 10261.79it/s]
89952it [00:09, 10468.13it/s]
91012it [00:09, 10096.40it/s]
92034it [00:09, 10073.66it/s]
93050it [00:09, 9609.88it/s] 
94187it [00:09, 9821.32it/s]
95

...

56896it [00:05, 9154.56it/s]
57982it [00:05, 9604.91it/s]
58976it [00:05, 9522.10it/s]
60057it [00:06, 9704.15it/s]
61043it [00:06, 9476.95it/s]
62158it [00:06, 9922.08it/s]
63164it [00:06, 9299.34it/s]
64284it [00:06, 9797.02it/s]
65561it [00:06, 10531.42it/s]
66721it [00:06, 10158.95it/s]
67834it [00:06, 10430.84it/s]
68896it [00:06, 9951.74it/s] 
69909it [00:07, 9704.35it/s]
70893it [00:07, 9683.01it/s]
71871it [00:07, 9503.76it/s]
73230it [00:07, 10442.28it/s]
74308it [00:07, 10146.09it/s]
75348it [00:07, 9608.71it/s] 
76396it [00:07, 9853.88it/s]
77588it [00:07, 10392.63it/s]
78747it [00:07, 10721.49it/s]
79837it [00:08, 10528.74it/s]
80977it [00:08, 10773.05it/s]
82065it [00:08, 10639.73it/s]
83137it [00:08, 10248.54it/s]
84352it [00:08, 10751.14it/s]
85440it [00:08, 9983.20it/s] 
86458it [00:08, 9513.99it/s]
87552it [00:08, 9900.91it/s]
88604it [00:08, 9915.88it/s]
89644it [00:08, 10051.11it/s]
90658it [00:09, 9527.48it/s] 
91623it [00:09, 9496.73it/s]
92581it [00:09, 8828.18it/

21411it [00:02, 9434.11it/s]
22448it [00:02, 9695.90it/s]
23592it [00:02, 10158.86it/s]
24782it [00:02, 10624.66it/s]
25862it [00:02, 9476.83it/s] 
27185it [00:02, 10353.45it/s]
28270it [00:03, 8677.83it/s] 
29218it [00:03, 7950.24it/s]
30085it [00:03, 8153.10it/s]
31058it [00:03, 8567.42it/s]
31955it [00:03, 8527.79it/s]
32982it [00:03, 8984.85it/s]
33911it [00:03, 9070.47it/s]
34837it [00:03, 8915.67it/s]
35903it [00:03, 9375.55it/s]
37203it [00:04, 10147.33it/s]
38356it [00:04, 10525.47it/s]
39433it [00:04, 10176.28it/s]
40471it [00:04, 9481.44it/s] 
41865it [00:04, 10484.35it/s]
42963it [00:04, 9233.80it/s] 
43999it [00:04, 9543.04it/s]
45118it [00:04, 9932.41it/s]
46314it [00:04, 10462.48it/s]
47394it [00:05, 10531.09it/s]
48471it [00:05, 10390.73it/s]
49527it [00:05, 10120.56it/s]
50596it [00:05, 10278.60it/s]
51634it [00:05, 10056.09it/s]
52648it [00:05, 9294.61it/s] 
53729it [00:05, 9701.72it/s]
54823it [00:05, 10042.37it/s]
55900it [00:05, 10248.69it/s]
56937it [00:06, 9625.29

92681it [00:09, 10269.80it/s]
93768it [00:09, 10441.88it/s]
95017it [00:10, 10974.63it/s]
96128it [00:10, 10790.70it/s]
97320it [00:10, 11104.65it/s]
98440it [00:10, 10663.67it/s]
100402it [00:10, 9459.91it/s]
 95%|█████████▌| 122/128 [27:24<01:18, 13.07s/it]
0it [00:00, ?it/s]
1141it [00:00, 11391.84it/s]
2050it [00:00, 10189.20it/s]
3132it [00:00, 9893.52it/s] 
4334it [00:00, 10446.78it/s]
5462it [00:00, 10527.91it/s]
6339it [00:00, 9573.56it/s] 
7190it [00:00, 9100.08it/s]
8301it [00:00, 9621.93it/s]
9223it [00:00, 9380.18it/s]
10134it [00:01, 8760.80it/s]
11213it [00:01, 9185.49it/s]
12292it [00:01, 9614.26it/s]
13325it [00:01, 9816.12it/s]
14473it [00:01, 10149.52it/s]
15494it [00:01, 9785.72it/s] 
16480it [00:01, 8721.98it/s]
17547it [00:01, 9221.78it/s]
18556it [00:01, 9465.43it/s]
19523it [00:02, 9438.98it/s]
20838it [00:02, 10310.49it/s]
21903it [00:02, 10135.09it/s]
23139it [00:02, 10644.29it/s]
24228it [00:02, 10212.53it/s]
25270it [00:02, 10096.76it/s]
26403it [00:02, 10426

56429it [00:05, 9132.68it/s]
57682it [00:05, 9939.30it/s]
58937it [00:06, 10600.84it/s]
60064it [00:06, 10792.64it/s]
61168it [00:06, 10754.66it/s]
62261it [00:06, 9531.55it/s] 
63252it [00:06, 8041.49it/s]
64123it [00:06, 8130.94it/s]
64983it [00:06, 8197.01it/s]
65836it [00:06, 8105.72it/s]
66845it [00:06, 8559.57it/s]
67724it [00:07, 8367.15it/s]
68863it [00:07, 9090.69it/s]
69903it [00:07, 9445.65it/s]
70873it [00:07, 9202.48it/s]
71812it [00:07, 8780.65it/s]
73181it [00:07, 9836.79it/s]
74218it [00:07, 9246.12it/s]
75298it [00:07, 9662.71it/s]
76484it [00:07, 10230.78it/s]
77797it [00:08, 10954.37it/s]
78931it [00:08, 9657.36it/s] 
79957it [00:08, 9567.67it/s]
80953it [00:08, 8494.85it/s]
81852it [00:08, 8506.34it/s]
82869it [00:08, 8943.55it/s]
83821it [00:08, 9109.00it/s]
84931it [00:08, 9626.44it/s]
86154it [00:08, 10282.50it/s]
87412it [00:09, 10877.19it/s]
88530it [00:09, 10745.12it/s]
89626it [00:09, 10481.27it/s]
90691it [00:09, 9436.49it/s] 
91666it [00:09, 9078.09it/s]
92

23566it [00:02, 9648.68it/s]
24850it [00:02, 10425.84it/s]
25926it [00:02, 9428.36it/s] 
26911it [00:02, 9496.53it/s]
27890it [00:02, 9394.99it/s]
28851it [00:02, 8835.15it/s]
29861it [00:03, 9169.68it/s]
30797it [00:03, 9150.64it/s]
31906it [00:03, 9656.75it/s]
32990it [00:03, 9911.75it/s]
34203it [00:03, 10474.36it/s]
35269it [00:03, 9775.60it/s] 
36496it [00:03, 10238.83it/s]
37758it [00:03, 10849.67it/s]
38868it [00:03, 9420.36it/s] 
39861it [00:04, 9417.70it/s]
40896it [00:04, 9675.44it/s]
41891it [00:04, 9048.10it/s]
42823it [00:04, 8123.08it/s]
43949it [00:04, 8863.42it/s]
44881it [00:04, 8832.21it/s]
45797it [00:04, 8836.65it/s]
46806it [00:04, 9175.09it/s]
47743it [00:05, 7700.96it/s]
48796it [00:05, 8375.06it/s]
49689it [00:05, 8494.99it/s]
50611it [00:05, 8696.34it/s]
51509it [00:05, 8635.32it/s]
52534it [00:05, 9063.41it/s]
53460it [00:05, 8955.69it/s]
54645it [00:05, 9559.04it/s]
55837it [00:05, 10162.58it/s]
56879it [00:05, 10001.03it/s]
57938it [00:06, 10160.89it/s]
5896

In [13]:
type(os.path.join(DATA_PATH, f"{i}.pkl"))

str

In [22]:
line = torch.IntTensor(tokenizer.convert_tokens_to_ids(json.loads(line)))







14723it [00:31, 956.50it/s]

In [ ]:
DATA_PATH = "SIMPLE_DATA"
os.makedirs(DATA_PATH, exist_ok=True)

In [ ]:
TOTAL_NUM_SECTORS = 8

In [ ]:
for i in range(TOTAL_NUM_SECTORS):
    with open(os.path.join(DATA_PATH, f"{i}.jsonl"), 'wb') as f_write:
        # for each sector
        for j in tqdm.tqdm(range(i, NUM_THREADS, TOTAL_NUM_SECTORS)):
            with open(f"cache/{j}_1.jsonl", "rb") as f_read:
                shutil.copyfileobj(f_read, f_write)

### Test

In [ ]:
with open("cache/0_1.jsonl", "r") as f:
    data = f.readline()
    data = f.readline()

In [ ]:
data

In [ ]:
!ray stop